In [ ]:
#LLM Chain
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import LLMChain
# Creating a simple prompt template with placeholders
template = PromptTemplate(
                           template="Tell me a joke about {content}.",
                           input_variables=["content"]
                           )
# Call the Llama model using CTransformer
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGUF", model_type="llama")
# Create an LLMChain object with the specified language model and prompt
chain = LLMChain(llm=llm, prompt=template, verbose=True)
# Run the chain with an input and print the output
print(chain.run(content="Math"))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a joke about Math.

> Finished chain.
его de 10.

My pleasure, here's a math-related joke for you:

Why did the number 5 go to therapy?

Because it was feeling insecure and wanted to work on its self-esteem!


In [ ]:
#Simple Sequential chain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

# Call the Llama model
llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGUF", model_type="llama")

# This is an LLMChain to write the first chain.
first_prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# This is an LLMChain to write the second chain
second_prompt = ChatPromptTemplate.from_template("Write a 20 words description for the following company:{company_name}")
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combining two chains in a simple sequential flow
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)
# Run the chain with an input and print the output
overall_simple_chain.run("gaming laptop")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new SimpleSequentialChain chain...

 Unterscheidung between different types of companies 
Company A: This company specializes in creating high-end gaming laptops with the latest hardware and cutting-edge designs. They cater to professional gamers and serious enthusiasts who want the best performance possible. Name suggestions could include "Gamer's Edge," "PowerPlay," or "GameChanger."
Company B: This company focuses on creating affordable gaming laptops that are accessible to a wider range of consumers. They prioritize practicality and durability, while still offering good performance. Name suggestions could include "GameOn," "PulseGear," or "FusionLaptop."
Company C: This company specializes in creating custom gaming laptops for individual customers. They offer a wide range of configurations and designs to suit the customer's preferences, while also providing excellent after-sales support. Name suggestions could include "CustomCraze," "GamersGarage," or "TechXtreme."
Com

' and cutting-edge. Name suggestions could include "EdgeOfInnovation," "NextGenGaming," or "FuturaMech."'

In [ ]:
#Sequential Chain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGUF", model_type="llama")

Review = '''The portable computers GamersTech impresses with its exceptional performance and elegant design.
        Its configuration of robust materials, a keyboard RVB personalisable and an effective cooling system, establishes a perfect
        balance between gaming performance and portability.'''

# prompt template 1: translate to English
first_prompt = ChatPromptTemplate.from_template("Convert in lower case:""\n\n{Review}")
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,output_key="English_Review" )

# prompt template 2: Summarize the English review
second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence:""\n\n{English_Review}")
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,output_key="summary")

# prompt template 3: translate to English
third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,output_key="language")

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template("Write a follow up response to the following "
                                                  "summary in the specified language:"
                                                  "\n\nSummary: {summary}\n\nLanguage: {language}"
                                                )
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


overall_chain = SequentialChain(
                                chains=[chain_one, chain_two, chain_three, chain_four],
                                input_variables=["Review"],
                                output_variables=["English_Review", "summary","followup_message"],
                               verbose=True
                              )
overall_chain(Review)







Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'The portable computers GamersTech impresses with its exceptional performance and elegant design. \n        Its configuration of robust materials, a keyboard RVB personalisable and an effective cooling system, establishes a perfect \n        balance between gaming performance and portability.',
 'English_Review': " \n        The laptop is powered by the latest Intel Core i7 processor with an enormous amount of memory (16 GB) and storage capacity (1 TB SSD), allowing for seamless multitasking and lightning-fast load times.\n        But what really sets GamersTech apart is its sleek and stylish design, which makes it look as good as it performs. The laptop features a slim and lightweight chassis with a durable anodized aluminum body that can withstand the rigors of daily use without sacrificing style.\n        Whether you're gaming on the go or working on a document, GamersTech has got you covered. Its advanced cooling system ensures that your laptop remains cool and quiet eve

In [12]:
#Router chain
# Importing necessary modules and classes
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain_community.llms import CTransformers

llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGUF", model_type="llama")

# Defining personas for router decision-making
math_teacher = """Your name is Arun, you are a Math teacher...{input}"""
spanish_teacher = """Your name is Ankit, you are a Spanish teacher...{input}"""
calculus_teacher = """Your name is Rohan, you are a Calculus teacher...{input}"""
# Setting up prompts for each person
prompt_infos = [
{"name": "Math Teacher", "description": "Good for answering questions about Math","prompt_template": math_teacher},
{"name": "Spanish Teacher", "description": "Good for answering questions about Spanish","prompt_template": spanish_teacher},
{"name": "Calculus Teacher", "description": "Good for answering questions about Calculus","prompt_template": calculus_teacher}
                 ]

# Mapping inputs to specific teacher chains
destination_chains = {}
for prompt_info in prompt_infos:
    prompt = PromptTemplate(template=prompt_info["prompt_template"], input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[prompt_info["name"]] = chain
# Default chain for general conversations
default_chain = ConversationChain(llm=llm, output_key="text")
# Preparing descriptions for router options
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# Setting up router chain template
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(template=router_template, input_variables=["input"],
output_parser=RouterOutputParser())
# Creating the main router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)
# Combining all chains into MultiPromptChain
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains,
default_chain=default_chain, verbose=True)

# Testing the router chain with various inputs
print(chain.run("What is the meaning of average?"))
print(chain.run("What is the derivative of xdx"))
print(chain.run("Translate 'Hello world' to Spanish"))


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new MultiPromptChain chain...
Math Teacher: {'input': 'What is the meaning of average?'}
> Finished chain.

 Unterscheidung between mean and median and mode.
Average, mean, median, and mode are all different ways of describing a set of data. Here's a breakdown of each:

* Mean: The mean is the sum of all the values in a dataset divided by the total number of values. It represents the "average" value in the dataset. For example, if you have a list of exam scores for 10 students, the mean score would be (80 + 72 + 63 + 54 + 46 + 35 + 29 + 24 + 18) / 10 = 54.
* Median: The median is the value in the middle of a dataset when the values are arranged in order. If there are an odd number of values, the median is the middle value. If there are an even number of values, the median is halfway between the two middle values. For example, if you have a list of exam scores for 10 students, the median score would be 54 (since there are 10 values).
* Mode: The mode is the value that appea

Math Teacher: {'input': 'What is the derivative of x dx'}
> Finished chain.
 + 2x^2 dy ?
 Einzeln 10.2013.

Answered:

The given expression is:

$$\frac{x}{dx} + 2x^2 \frac{dy}{dx}$$

To find the derivative, we can use the chain rule:

$$\frac{x}{dx} = 1$$

$$2x^2 \frac{dy}{dx} = 2x \frac{dy}{dx} + 2x \frac{dx}{dx}$$

Now, we have:

$$= 1 + 2x \frac{dy}{dx} + 2x \frac{dx}{dx}$$

$$= 1 + 2x \frac{dy}{dx} + 0$$

So, the derivative of x dx + 2x^2 dy is 1.


> Entering new MultiPromptChain chain...


Spanish Teacher: {'input': 'Hello world'}
> Finished chain.
!
iellement I am passionate about teaching and sharing my knowledge of the Spanish language with others. I have been teaching for several years now, and I have to say that it's one of the most rewarding professions out there. There's nothing quite like seeing the light bulb go off in a student's head when they finally understand a concept or master a new verb tense.
However, I also know that teaching can be challenging at times, especially when it comes to keeping students engaged and motivated. That's why I always try to come up with creative and fun ways to teach Spanish, whether it's through games, role-playing exercises, or cultural events.
I believe that the key to success in teaching is to be able to connect with your students on a personal level. By building trust and rapport, you can create a safe and supportive learning environment where students feel comfortable taking risks and trying new things. And let's be real, 